In [2]:
import os
import django

os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'quanta.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

In [3]:
from apps.organization.models import Organization, OrganizationUser, OrganizationOwner

In [11]:
from apps.orders.models import Facturation, Commercial
from apps.inventories.models import Product

from apps.organization.models import Organization
from django.db.models import Count, F, Func, Q, Sum, Value

In [5]:
from apps.orders.models import Facturation

In [6]:
user = OrganizationUser.objects.first().user

In [7]:
organizations = OrganizationUser.objects.for_user(user=user).values_list("organization_id")

In [8]:
organizations

<QuerySet [(UUID('1688cf3e-95c4-4f43-b37d-361e77e37025'),), (UUID('57badfd7-c2f9-431a-a627-1b77f13669ee'),), (UUID('ae262ba8-490f-4316-a71a-5331ff43e26a'),), (UUID('ebdcb44f-c74d-4474-9a08-e368677cb3dd'),), (UUID('f549aac5-4220-457d-84b6-383271896b6d'),)]>

In [9]:
organizations

<QuerySet [(UUID('1688cf3e-95c4-4f43-b37d-361e77e37025'),), (UUID('57badfd7-c2f9-431a-a627-1b77f13669ee'),), (UUID('ae262ba8-490f-4316-a71a-5331ff43e26a'),), (UUID('ebdcb44f-c74d-4474-9a08-e368677cb3dd'),), (UUID('f549aac5-4220-457d-84b6-383271896b6d'),)]>

In [10]:
Facturation.objects.for_organization(organization=organizations).count()

9

In [27]:


products = Product.objects.for_organization(organization=organizations).all()

In [28]:
product_reports = products.values("organization_id", "name", "quantity").annotate(
            total_facturation_price=F("facturation_price") * F("quantity"),
            total_purchase_price=F("purchase_price") * F("quantity"),
            total_profit=F("total_facturation_price") - F("total_purchase_price"),
)

In [29]:
for product in product_reports:
    print(product)

{'organization_id': UUID('57badfd7-c2f9-431a-a627-1b77f13669ee'), 'name': 'Vetements', 'total_facturation_price': Decimal('9000000'), 'total_purchase_price': Decimal('1500000'), 'total_profit': Decimal('7500000')}
{'organization_id': UUID('57badfd7-c2f9-431a-a627-1b77f13669ee'), 'name': 'Marmite', 'total_facturation_price': Decimal('810000'), 'total_purchase_price': Decimal('81000'), 'total_profit': Decimal('729000')}
{'organization_id': UUID('ae262ba8-490f-4316-a71a-5331ff43e26a'), 'name': 'Verre', 'total_facturation_price': Decimal('11000000'), 'total_purchase_price': Decimal('550000'), 'total_profit': Decimal('10450000')}
{'organization_id': UUID('ae262ba8-490f-4316-a71a-5331ff43e26a'), 'name': 'Monture (cadre)', 'total_facturation_price': Decimal('-18000'), 'total_purchase_price': Decimal('-9000'), 'total_profit': Decimal('-9000')}
{'organization_id': UUID('ae262ba8-490f-4316-a71a-5331ff43e26a'), 'name': 'Collyres', 'total_facturation_price': Decimal('0'), 'total_purchase_price': D